In [ ]:
import os
import json
import ssl
import traceback
import pandas as pd
import importlib
from aiohttp import web
from icecream import ic
import nest_asyncio
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from gsheet_handler import GSheetHandler

nest_asyncio.apply()

module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

# Extract example dialog from Google Sheet
example_dialog = ggl.extract_data_from_google_sheet(
    'https://docs.google.com/spreadsheets/d/1sBAFF62fAjSdPrGYTWgpjmBjLRiFCb8DNAX8ZypI-_U/edit?gid=0#gid=0',
    0
)

def create_training_prompt(dialogs):
    instruction_message = SystemMessage(content="Ты консультант по инвестиционным программам правительства Москвы.")
    examples_message_content = "Примеры диалогов:\n"
    
    for i, row in dialogs.iterrows():
        examples_message_content += f"Человек: {row['Human']}\nAI: {row['AI']}\n"

    examples_message = SystemMessage(content=examples_message_content)
    
    messages = [instruction_message, examples_message]
    prompt_template = ChatPromptTemplate.from_messages(messages)
    return prompt_template

# Create training prompt
training_prompt = create_training_prompt(example_dialog)
formatted_messages = training_prompt.format_messages()

for message in formatted_messages:
    print(f"{message.type}: {message.content}")

# Load config
with open('config.json') as config_file:
    config = json.load(config_file)

os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "nlmk_bot"

# Initialize ChatOpenAI model
chat = ChatOpenAI()

# History of messages
message_history = training_prompt.format_messages()

# Function to create a prompt with history
def create_prompt_with_history(history, user_input):
    messages = history + [HumanMessage(content=user_input)]
    return ChatPromptTemplate.from_messages(messages)

# Aiohttp web app
app = web.Application()

async def index(request):
    return web.FileResponse('./templates/index.html')

async def chat_endpoint(request):
    try:
        data = await request.json()
        user_message = data.get('message', '')
        ic(f"Received message: {user_message}")
        
        # Create prompt with history and user input
        prompt_with_history = create_prompt_with_history(message_history, user_message)
        formatted_messages = prompt_with_history.format_messages()
        response = chat.invoke(formatted_messages)
        ic(f"Generated response: {response.content}")
        
        # Add the new user message and AI response to the history
        message_history.append(HumanMessage(content=user_message))
        message_history.append(AIMessage(content=response.content))
        
        return web.json_response({'response': response.content})
    except Exception as e:
        error_message = f"Произошла ошибка: {e}"
        ic(error_message)
        traceback.print_exc()
        return web.json_response({'response': error_message}, status=500)

async def acme_challenge(request):
    filename = request.match_info['filename']
    return web.FileResponse(os.path.join('.well-known/acme-challenge', filename))

app.add_routes([web.get('/', index)])
app.add_routes([web.post('/chat', chat_endpoint)])
app.add_routes([web.get('/.well-known/acme-challenge/{filename}', acme_challenge)])

# Setup static file handling
app.router.add_static('/static/', path=str(os.path.join(os.getcwd(), 'static')), name='static')

if __name__ == "__main__":
    # Create an SSL context
    ssl_context = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH)
    ssl_context.load_cert_chain(certfile='fullchain.pem',
                                keyfile='privkey.pem')
    
    web.run_app(app, host='0.0.0.0', port=8080, ssl_context=ssl_context)


no docx
no docx
system: Ты консультант по инвестиционным программам правительства Москвы.
system: Примеры диалогов:
Человек: Интересует программа «Земля за 1 рубль для производства»
AI: Это программа для создания новых производств на территории города Москвы. По программе «Земля за 1 рубль для производства» арендная плата на весь срок аренды земельного участка, оформляемого для проектирования и строительства/реконструкции объектов промышленно-производственного назначения, устанавливается в размере 1 (одного) рубля в год. 
В случае возникновения дополнительных вопросов готовы переключить Вас на соответствующего специалиста.
Человек: Где найти более подробную информацию о конкретном земельном участке?
AI: Ознакомиться с земельными участками, потенциально доступными для строительства производственных объектов, можно на Инвестиционном портале Москвы. Для этого необходимо на главной странице Инвестиционного портала города Москвы investmoscow.ru (инвестмоскоу точка ру) выбрать пункт 4 «Земля

ic| f"Received message: {user_message}": 'Received message: Подобрать решение для бизнеса'
ic| f"Generated response: {response.content}": ('Generated response: Для подбора оптимального решения для вашего бизнеса, мне '
                                                'потребна более конкретная информация о вашем проекте, бизнесе и целях. Могли '
                                                'бы вы поделиться дополнительными деталями, например:
                                               '
                                                '
                                               '
                                                '1. Какова отрасль вашего бизнеса?
                                               '
                                                '2. Какие цели и задачи вы ставите перед своим бизнесом?
                                               '
                                                '3. На каком этапе находится ваш проект: развитие, запуск нового напр

In [ ]:
import importlib
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

#меры поддержки для бизнеса от правительства Москвы
support = ggl.extract_data_from_google_sheet(
    'https://docs.google.com/spreadsheets/d/1C3m50L3iUWhkkrp5CWEmksVRAkA8y4hX6m2tYgS8-4I/edit?gid=1891107571#gid=1891107571',
    0)
support

In [ ]:
pip install flask-talisman